In [1328]:
from scipy.io import loadmat
from scipy.io import savemat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MultipleLocator
import statistics
import csv
plt.rcParams['figure.dpi'] = 900
from matplotlib import font_manager
plt.style.use('_mpl-gallery')
import matplotlib as mpl
from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import TheilSenRegressor
from matplotlib import font_manager
mpl.rcParams['font.family'] = 'Times New Roman'
import scipy.io as sio
import os

def load_grant_table(file_path):
    tbs_lookup_table = '/Users/liuyuxin/Code/airscreen/data/sync airscreen dci and pkt/ul_whole_table/TB_size_look_up_table_use.xlsx'
    data_df = pd.read_excel(tbs_lookup_table, header=0)
    data_df = data_df.iloc[:, 1:]
    tbs_lookup_space = data_df.to_numpy(dtype=np.float64)

    grant_table = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        wrap_count = 0
        last_tti = None
        for row in reader:
            row = row[0].split('\t')
            raw_tti = int(row[4])*10+int(row[3])
            rv = int(row[6])
            if row[7] == "TBS_Index_0":
                continue
            tbs_field = row[7].split('_')[-1]
            tbs_index = int(tbs_field.rstrip('A'))
            nof_rb = int(row[8])
            tbs = tbs_lookup_space[tbs_index, nof_rb-1]//8
            if last_tti is not None and raw_tti < last_tti:
                wrap_count += 1
            adjusted_tti = wrap_count * 10240 + raw_tti
            last_tti = raw_tti
            grant_table.append([adjusted_tti, rv, tbs])
    return grant_table

def load_pusch_table(file_path):
    pusch_tx_report = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        wrap_count = 0
        last_tti = None
        for row in reader:
            row = row[0].split('\t')
            raw_tti = int(row[2])
            tbs = int(row[4])
            rv = int(row[3])
            if last_tti is not None and raw_tti < last_tti:
                wrap_count += 1
            adjusted_tti = wrap_count * 10240 + raw_tti
            last_tti = raw_tti
            pusch_tx_report.append([adjusted_tti, tbs, rv])
    return pusch_tx_report

def load_bsr_table(file_path):
    bsr_table = []
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        wrap_count = 0
        last_tti = None
        for row in reader:
            row = row[0].split('\t')
            raw_tti = int(row[3])*10+int(row[2])
            tbs = int(row[4])
            padding_size = int(row[5])
            bsr_type = int(row[7])
            lcg = int(row[8])
            if bsr_type == 1 and lcg != -1:
                bsr_index = int(row[10+lcg])
            elif bsr_type == 2 and lcg != -1:
                bsr_index = int(row[10+lcg])
            else:
                bsr_index = -1
            # bsr_index = int(row[8])
            if last_tti is not None and raw_tti < last_tti:
                wrap_count += 1
            adjusted_tti = wrap_count * 10240 + raw_tti
            last_tti = raw_tti
            bsr_table.append([adjusted_tti, tbs, padding_size, bsr_index])
    return bsr_table

In [1329]:
# grant_file = '../IMC2025/BS_congestion/ngscope_verify/0811_5/B16C_report.txt'
# pusch_file = '../IMC2025/BS_congestion/ngscope_verify/0811_5/B139_report.txt'
# bsr_file = '../IMC2025/BS_congestion/ngscope_verify/0811_5/B064_report.txt'

# # grant_table = load_grant_table(grant_file)
# pusch_table = load_pusch_table(pusch_file)
# bsr_table = load_bsr_table(bsr_file)

# # print(grant_table[0][0], grant_table[-1][0])
# # print(pusch_table[0][0], pusch_table[-1][0])
# # print(bsr_table[0][0], bsr_table[-1][0])

# ul_table_start_tti = min(pusch_table[0][0], bsr_table[0][0])
# ul_table_end_tti = max(pusch_table[-1][0], bsr_table[-1][0])
# ul_whole_table = [[0]*8 for _ in range(ul_table_start_tti, ul_table_end_tti+1)] # tti, pusch, pusch_tbs, pusch_rv, bsr, bsr_tbs, bsr_padding_size, bsr_index
# for i in range(len(ul_whole_table)):
#     ul_whole_table[i][0] = ul_table_start_tti + i
# with open("../IMC2025/BS_congestion/ngscope_verify/0811_5/ul_whole_table.txt", "w") as f:
#     for row in ul_whole_table:
#         tti = row[0]

#         for pusch_row in pusch_table:
#             if tti == pusch_row[0]:
#                 row[1] = 1
#                 row[2] = pusch_row[1]
#                 row[3] = pusch_row[2]
#                 break

#         for bsr_row in bsr_table:
#             if tti == bsr_row[0]:
#                 row[4] = 1
#                 row[5] = bsr_row[1]
#                 row[6] = bsr_row[2]
#                 row[7] = bsr_row[3]
#                 break

#         row_str = "\t".join(str(elem) for elem in row)
#         f.write(row_str + "\n")

In [ ]:
# ul_table_file = '../IMC2025/BS_congestion/ngscope_verify/0811_5/ul_whole_table.txt'
# offset_value_dci = 12226+8
# offset_value_ul = 9014+13456+8
# tx_end = 756260

# ul_table = []
# with open(ul_table_file, "r") as file:
#     for line in file:
#         parts = line.strip().split('\t')
#         row = [float(p) if '.' in p else int(p) for p in parts]
#         ul_table.append(row)
# # tti, pusch, pusch_tbs, pusch_rv, bsr, bsr_tbs, bsr_padding_size, bsr_index
# ul_table = [[row[0] - offset_value_ul] + row[1:] for row in ul_table]
# ul_table = [row for row in ul_table if 0 <= row[0] <= tx_end]

# dci_file = '../IMC2025/BS_congestion/ngscope_verify/0811_5/dci_raw_log_ul_freq_2127500000.dciLog'
# rnti_ue = 6153
# dci_table = []
# with open(dci_file, "r") as file:
#     last_tti = None
#     wrap_count = 0
#     for line in file:
#         parts = line.strip().split('\t')
#         row = [float(p) if '.' in p else int(p) for p in parts]
#         raw_tti = row[0]
#         if last_tti is not None and raw_tti < last_tti:
#                 wrap_count += 1
#         adjusted_tti = wrap_count * 10240 + raw_tti
#         last_tti = raw_tti
#         row[0] = adjusted_tti
#         dci_table.append(row)
# dci_table = [[row[0] - offset_value_dci] + row[1:] for row in dci_table]
# dci_table = [row for row in dci_table if 0 <= row[0] <= tx_end]
# # dci_ue = [row for row in dci_table if row[1] == rnti_ue]

# load_table = [[row[0], row[2]] for row in dci_table]
# load_table = list({(r[0], r[1]) for r in load_table})
# # load_table = sorted([[r[0], min(r[1], 75)] for r in load_table], key=lambda x: x[0])
# load_table = sorted([[r[0], min(r[1], 60)] for r in load_table], key=lambda x: x[0])

# sio.savemat('../IMC2025/BS_congestion/ngscope_verify/0811_5/ul_load_cache.mat', {
#         "ul_table": ul_table,
#         "load_table": load_table})

In [1331]:
MAT_FILE = '../IMC2025/BS_congestion/ngscope_verify/0811_5/ul_load_cache.mat'
if os.path.exists(MAT_FILE):
    data = sio.loadmat(MAT_FILE)
    ul_table = data["ul_table"].tolist()
    load_table = data["load_table"].tolist()

pkt_size_array = [40,40,40,40,40,40,100,200,
                  200,200,300,600,630,600,700,800,900,1000,
                  900,900,970,960,1030,1000,1060,1120,1200,1250,
                  1240,1200,1140,1230,1130]
interpkt_delay_array = [4000,3000,2000,1700,1300,1000,1400,2000,
                        1500,800,800,1200,1000,800,800,800,800,800,
                        650,600,600,550,550,500,500,500,500,500,
                        450,400,350,350,300]
rate_labels = [
    f"{(size * 8 * 1000) // delay}"
    for size, delay in zip(pkt_size_array, interpkt_delay_array)
]

index_list = list(range(64))
lower_bounds = [ 0,  0, 10, 12, 14, 17, 19, 22, 26, 31, 36, 42, 49, 57, 67, 78,
    91,107,125,146,171,200,234,274,321,376,440,515,603,706,826,967,
   1132,1326,1552,1817,2127,2490,2915,3413,3995,4677,5476,6411,7505,
   8787,10287,12043,14099,16507,19325,22624,26487,31009,36304,42502,
   49759,58255,68201,79846,93479,109439,128125, 150000]
upper_bounds = [ 0, 10, 12, 14, 17, 19, 22, 26, 31, 36, 42, 49, 57, 67, 78, 91,
   107,125,146,171,200,234,274,321,376,440,515,603,706,826,967,1132,
   1326,1552,1817,2127,2490,2915,3413,3995,4677,5476,6411,7505,8787,
  10287,12043,14099,16507,19325,22624,26487,31009,36304,42502,49759,
  58255,68201,79846,93479,109439,128125,150000, 150000]

periods = [[0,20028],[22992, 43030],[45992, 66039],[68982, 89030],[92002, 112040],[115002, 135043],[138002, 158050],[160992, 181145],
           [183992, 204045],[206992, 227053],[230002, 250045],[253002, 273060],[276002, 296045],[298962, 319195],[322002, 342050],[345002, 365055],[368002, 388050],[390992, 411050],
           [414012, 434050],[436992, 457050],[460012, 480050],[483012, 503058],[506012, 526058],[529002, 549065],[552002, 572060],[575002, 595055],[598002, 618060],[621012, 641333],
           [644012, 664055],[667002, 687145],[690012, 710225],[713012, 733220],[736042, 756260]]

In [1332]:
def calculate_ratio_load(part_index, load_table, ul_table):
    bs_load = []

    requested_resources_upper = []
    requested_resources_mid = []
    requested_resources_lower = []

    allocated_resources = []
    ratio_update = []

    period_start = periods[part_index][0]
    period_end = periods[part_index][-1]

    for j in range(len(load_table)):
        tti = load_table[j][0]
        if tti >= period_start and tti <= period_end:
            # bs_load.append([tti, load_table[j][1]/75])
            bs_load.append([tti, load_table[j][1]/60])
        if tti > period_end:
            break
    
    nof_bsr = 0
    for k in range(len(ul_table)):
        tti = ul_table[k][0]
        if tti >= period_start and tti <= period_end:
            if ul_table[k][4] == 1 and ul_table[k][7] != -1:
                nof_bsr += 1
        if tti > period_end:
            break
    
    bsr_index = 0
    for m in range(len(ul_table)):
        tti = ul_table[m][0]
        if tti > period_end:
            break
        if tti >= period_start and tti <= period_end:
            if ul_table[m][4] == 1 and ul_table[m][7] != -1:
                upper_requested_bytes = 0
                lower_requested_bytes = 0
                allocated_bytes = 0
                overlap = 0

                bsr_index += 1

                if bsr_index == nof_bsr: # The last BSR
                    for a in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0]-ul_table[m][0]
                        if tti_diff > 0 and tti_diff <= 4 and ul_table[a][3] == 0:
                            overlap += ul_table[a][2]
                        if tti_diff > 4:
                            break
                    upper_value = upper_bounds[ul_table[m][7]]
                    lower_value = lower_bounds[ul_table[m][7]]

                    if overlap >= lower_value:
                            upper_requested_bytes += upper_value
                            lower_requested_bytes += lower_value
                    else:
                        upper_requested_bytes += (upper_value - overlap)
                        lower_requested_bytes += (lower_value - overlap)
                    
                    for b in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0] - ul_table[m][0]
                        if tti_diff >= 5 and ul_table[b][3] == 0:
                            allocated_bytes += ul_table[b][2]
                    continue

                else:
                    for a in range(m, len(ul_table)):
                        tti_diff = ul_table[a][0]-ul_table[m][0]
                        if tti_diff > 0 and tti_diff <= 4 and ul_table[a][3] == 0:
                            overlap += ul_table[a][2]
                        if tti_diff > 4:
                            break
                    upper_value = upper_bounds[ul_table[m][7]]
                    lower_value = lower_bounds[ul_table[m][7]]

                    if overlap >= lower_value:
                        upper_requested_bytes += upper_value
                        lower_requested_bytes += lower_value
                    else:
                        upper_requested_bytes += (upper_value - overlap)
                        lower_requested_bytes += (lower_value - overlap)
                    
                    for c in range(m + 1, len(ul_table)):
                        if ul_table[c][4] == 1 and ul_table[c][7] != -1:
                            bsr_period = ul_table[c][0] - ul_table[m][0]
                            break

                    for b in range(m, len(ul_table)):
                        tti_diff = ul_table[b][0] - ul_table[m][0]
                        if tti_diff >= 5 and tti_diff <= (bsr_period+4) and ul_table[b][3] == 0:
                            allocated_bytes += ul_table[b][2]
                        if tti_diff > (bsr_period+4):
                            break

                requested_resources_upper.append(upper_requested_bytes)
                requested_resources_lower.append(lower_requested_bytes)
                mid_requested_bytes = (lower_requested_bytes + upper_requested_bytes)//2
                requested_resources_mid.append(mid_requested_bytes)
                allocated_resources.append(allocated_bytes)
                # ratio_update.append([tti, allocated_bytes/(upper_requested_bytes*(ul_table[m][7] ** 0.5 ))]) if upper_requested_bytes != 0 else ratio_update.append([tti, 5])
                # ratio_update.append([tti, allocated_bytes/(upper_requested_bytes*ul_table[m][7])]) if upper_requested_bytes != 0 else ratio_update.append([tti, 50])
                ratio_update.append([tti, allocated_bytes/upper_requested_bytes]) if upper_requested_bytes != 0 else ratio_update.append([tti, 5])
                # ratio_update.append([tti, allocated_bytes/lower_requested_bytes]) if lower_requested_bytes != 0 else ratio_update.append([tti, 5])
    return bs_load, ratio_update

In [1333]:
part_list = [ elem for elem in range(33)]

for i in range(len(part_list)):
    part_index = part_list[i]
    bs_load, ratio_update = calculate_ratio_load(i, load_table, ul_table)
    bs_load_avg = []
    for j in range(len(ratio_update)-1):
        bsr_current = ratio_update[j][0]
        bsr_next = ratio_update[j+1][0]
        load_list = []
        for m in range(len(bs_load)):
            if bs_load[m][0]-4 > bsr_current and bs_load[m][0]-4 <= bsr_next:
                load_list.append(bs_load[m][1])
            if bs_load[m][0]-4 > bsr_next:
                break
        # bs_load_avg.append([bsr_current, np.mean(load_list)]
        # bs_load_avg.append([bsr_next, np.mean(load_list)])
        if all(x == 0 for x in load_list):
            bs_load_avg.append([bsr_current, 0])
        else:
            non_zero_elements = [x for x in load_list if x != 0]
            bs_load_avg.append([bsr_current, np.mean(non_zero_elements)])
    
    print(f"Processing rate {rate_labels[part_index]} kbps, pkt size {pkt_size_array[part_index]} bytes, inter-pkt delay {interpkt_delay_array[part_index]} us")
    # print(ratio_update[-10:], bs_load_avg[-10:])


    ############################# Method: average in time window ##################################
    # smoothed_window = 100
    # bs_load_avg_smoothed = []
    # ratio_update_smoothed = []
    # time_points = []
    # start_time = bs_load_avg[0][0]
    # end_time = bs_load_avg[-1][0]
    # curr_t = start_time

    # while curr_t + smoothed_window <= end_time:
    #     load_values = [v[1] for v in bs_load_avg if curr_t <= v[0] < curr_t + smoothed_window]
    #     ratio_values = [v[1] for v in ratio_update if curr_t <= v[0] < curr_t + smoothed_window]

    #     if load_values:
    #         bs_load_avg_smoothed.append(sum(load_values) / len(load_values))
    #     else:
    #         bs_load_avg_smoothed.append(0)

    #     if ratio_values:
    #         ratio_update_smoothed.append(sum(ratio_values) / len(ratio_values))
    #     else:
    #         ratio_update_smoothed.append(0)

    #     time_points.append(curr_t + smoothed_window // 2)
    #     curr_t += smoothed_window

    ############################ Method: smoothed with history mean ##################################
    smoothed_window = 20
    alpha = 0.6
    bs_load_avg_smoothed = []
    ratio_update_smoothed = []
    time_points = []

    for k in range(len(bs_load_avg)):
        tti = bs_load_avg[k][0]
        load = bs_load_avg[k][1]
        ratio = ratio_update[k][1]

        # Compute history mean: ratios in (tti - 50ms, tti)
        history_ratio_values = [ratio_update[m][1] for m in range(k) if bs_load_avg[m][0] > tti - smoothed_window]
        history_load_values = [bs_load_avg[m][1] for m in range(k) if bs_load_avg[m][0] > tti - smoothed_window]
        
        if not history_ratio_values:
            ratio_update_smoothed.append(ratio)  # no history, keep original
        else:
            history_mean = sum(history_ratio_values) / len(history_ratio_values)
            new_r = alpha * ratio + (1 - alpha) * history_mean
            ratio_update_smoothed.append(new_r)

        if not history_load_values:
            bs_load_avg_smoothed.append(ratio)  # no history, keep original
        else:
            history_mean = sum(history_load_values) / len(history_load_values)
            new_l = alpha * load + (1 - alpha) * history_mean
            bs_load_avg_smoothed.append(new_l)

        time_points.append(tti)


    # filename = f'../IMC2025/BS_congestion/ngscope_verify/0811_5/Points_size_{pkt_size_array[part_index]}_delay_{interpkt_delay_array[part_index]}.txt'
    # with open(filename, 'w') as f:
    #     for t, x, y in zip([row[0] for row in bs_load_avg[2:]], [row[1] for row in bs_load_avg[2:]], [row[1] for row in ratio_update][:-3]):
    #     # for t, x, y in zip([row[0] for row in bs_load_avg[:]], [row[1] for row in bs_load_avg[:]], [row[1] for row in ratio_update][:-1]):
    #     # for t, x, y in zip([row[0] for row in bs_load_avg[1:]], [row[1] for row in bs_load_avg[1:]], [row[1] for row in ratio_update][:-2]):
    #     # for t, x, y in zip([row[0] for row in bs_load_avg[5:]], [row[1] for row in bs_load_avg[5:]], [row[1] for row in ratio_update][:-6]):
    #         f.write(f"{t}\t{x}\t{y}\n")

    filename = f'../IMC2025/BS_congestion/ngscope_verify/0811_5/SmoothPoints_size_{pkt_size_array[part_index]}_delay_{interpkt_delay_array[part_index]}.txt'
    with open(filename, 'w') as f:
        for t, x, y in zip(time_points, bs_load_avg_smoothed, ratio_update_smoothed):
        # for t, x, y in zip([row[0] for row in bs_load_avg[:]], [row[1] for row in bs_load_avg[:]], [row[1] for row in ratio_update][:-1]):
            f.write(f"{t}\t{x}\t{y}\n")

Processing rate 80 kbps, pkt size 40 bytes, inter-pkt delay 4000 us
Processing rate 106 kbps, pkt size 40 bytes, inter-pkt delay 3000 us
Processing rate 160 kbps, pkt size 40 bytes, inter-pkt delay 2000 us
Processing rate 188 kbps, pkt size 40 bytes, inter-pkt delay 1700 us
Processing rate 246 kbps, pkt size 40 bytes, inter-pkt delay 1300 us
Processing rate 320 kbps, pkt size 40 bytes, inter-pkt delay 1000 us
Processing rate 571 kbps, pkt size 100 bytes, inter-pkt delay 1400 us
Processing rate 800 kbps, pkt size 200 bytes, inter-pkt delay 2000 us
Processing rate 1066 kbps, pkt size 200 bytes, inter-pkt delay 1500 us
Processing rate 2000 kbps, pkt size 200 bytes, inter-pkt delay 800 us
Processing rate 3000 kbps, pkt size 300 bytes, inter-pkt delay 800 us
Processing rate 4000 kbps, pkt size 600 bytes, inter-pkt delay 1200 us
Processing rate 5040 kbps, pkt size 630 bytes, inter-pkt delay 1000 us
Processing rate 6000 kbps, pkt size 600 bytes, inter-pkt delay 800 us
Processing rate 7000 kbp

In [1334]:
# part_start = 0 ######### ratio > 2
# part_end = 4

# part_start = 5 ######### ratio > 1
# part_end = 15

# part_start = 5  
# part_end = 27

part_start = 0
part_end = 33


bs_load_avg = []
ratio_update = []
for i in range(part_start, part_end):
    file = f'../IMC2025/BS_congestion/ngscope_verify/0811_5/SmoothPoints_size_{pkt_size_array[i]}_delay_{interpkt_delay_array[i]}.txt'
    with open(file, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader)
        for row in reader:
            tti = int(row[0])
            bs_load_avg.append(float(row[1]))
            ratio_update.append(float(row[2]))

plt.figure(figsize=(12, 8))
H, xedges, yedges = np.histogram2d(
    bs_load_avg,
    ratio_update,
    bins=[50, 100],
    range=[[0, 1], [0, 2]]
    # bs_load_avg_smoothed, 
    # ratio_update_smoothed,
    # bins=[50, 50],
    # range=[[0, max(bs_load_avg_smoothed)], [0, 2]]
)

vmax = np.percentile(H[H > 0], 95)
cmap = plt.cm.get_cmap('viridis').copy()
mesh = plt.pcolormesh(
    xedges,
    yedges,
    H.T,
    cmap=cmap,
    # vmin=vmin,
    vmax=vmax,
    shading='auto'
)

cb = plt.colorbar(mesh)
cb.set_label('Point Density', fontsize=16)
cb.ax.tick_params(labelsize=14)

plt.xlabel('Base station load', fontsize=20)
plt.ylabel('Ratio', fontsize=20)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.grid(True, linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.xlim(left=0)
plt.ylim(0, 2)
# plt.ylim(1e-2, 2)
# plt.yscale('log')
plt.xlim(0, 1)

rate_labels = [
    f"{(size * 8 * 1000) // delay}"
    for size, delay in zip(pkt_size_array[part_start:part_end], interpkt_delay_array[part_start:part_end])
]
# plt.title(f'Sending rate {rate_labels[0]} kbps', fontsize=20, fontweight='bold')
plt.savefig('/Users/liuyuxin/Desktop/heatmap.pdf', dpi=900)
plt.show()

/var/folders/6m/14gw8zf15q71l57fp8pmwpxm0000gn/T/ipykernel_59871/4066144634.py:39: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('viridis').copy()
